In [39]:
import torch
from transformers import AutoModel
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import transformers
from datasets import load_dataset
import handler

import pathlib
import sklearn
import datasets
import pandas as pd
import glob
import functools
import os
import sys

TOKENIZERS_PARALLELISM=False

In [53]:
!ls ./trainedmodels/


config.json             model_store             special_tokens_map.json
config.properties       model_weights.pt        tokenizer.json
index_to_name.json      requirements.txt        tokenizer_config.json
model.safetensors       setup_config.json       vocab.txt


In [61]:
BASE_DIR = '/Users/tylerklimas/Desktop/BERTModel/TrainedModels'

MODEL_NAME = 'pt_classifier'
MODEL_VERSION = '1.0'
MODEL_STORE='${BASE_DIR}/model_store'
MODEL_SERIALIZED_FILE="${BASE_DIR}/model_weights.pt"

TOKENIZER_FILES= "${BASE_DIR}/tokenizer_config.json, ${BASE_DIR}/special_tokens_map.json, ${BASE_DIR}/vocab.txt,  ${BASE_DIR}/tokenizer.json"
MODEL_EXTRA_FILES="${BASE_DIR}/setup_config.json, ${TOKENIZER_FILES}"


In [62]:
%%bash

BASE_DIR='/Users/tylerklimas/Desktop/BERTModel/TrainedModels'

MODEL_NAME='pt_classifier'
MODEL_VERSION='1.0'
MODEL_STORE="${BASE_DIR}/model_store"
MODEL_SERIALIZED_FILE="${BASE_DIR}/model_weights.pt"
REQUIREMENTS_FILE="${BASE_DIR}/requirements.txt" # Path to your requirements.txt

# Ensure no spaces after commas
TOKENIZER_FILES="${BASE_DIR}/tokenizer_config.json,${BASE_DIR}/special_tokens_map.json,${BASE_DIR}/vocab.txt,${BASE_DIR}/tokenizer.json"
MODEL_EXTRA_FILES="${BASE_DIR}/index_to_name.json,${BASE_DIR}/setup_config.json,${TOKENIZER_FILES}"

mkdir -p $MODEL_STORE

torch-model-archiver --model-name ${MODEL_NAME} \
--version ${MODEL_VERSION} \
--serialized-file ${MODEL_SERIALIZED_FILE} \
--export-path ${MODEL_STORE} \
--extra-files "${MODEL_EXTRA_FILES}" \
--handler ./handler.py \
--requirements-file ${REQUIREMENTS_FILE} \
-f



WARNING - Overwriting /Users/tylerklimas/Desktop/BERTModel/TrainedModels/model_store/pt_classifier.mar ...


In [63]:
!pwd

/Users/tylerklimas/Desktop/BERTModel


In [64]:
%%bash --bg
torchserve --ts-config /Users/tylerklimas/Desktop/BERTModel/TrainedModels/config.properties \
--start --model-store /Users/tylerklimas/Desktop/BERTModel/TrainedModels/model_store --ncs

In [68]:
%%bash 
torchserve --stop

TorchServe has stopped.


In [65]:
!cat /Users/tylerklimas/Desktop/BERTModel/TrainedModels/config.properties

load_models=all
inference_address=http://0.0.0.0:9080
management_address=http://0.0.0.0:9081
metrics_address=http://0.0.0.0:9082
model_store=model_store
async_logging=true

In [66]:
!curl "http://localhost:9081/models/pt_classifier"

[
  {
    "modelName": "pt_classifier",
    "modelVersion": "1.0",
    "modelUrl": "pt_classifier.mar",
    "runtime": "python",
    "minWorkers": 8,
    "maxWorkers": 8,
    "batchSize": 1,
    "maxBatchDelay": 100,
    "loadedAtStartup": true,
    "workers": [
      {
        "id": "9000",
        "startTime": "2024-03-10T10:56:02.022Z",
        "status": "UNLOADING",
        "memoryUsage": 0,
        "pid": 78925,
        "gpu": false,
        "gpuUsage": "N/A"
      },
      {
        "id": "9001",
        "startTime": "2024-03-10T10:56:02.025Z",
        "status": "UNLOADING",
        "memoryUsage": 0,
        "pid": 78928,
        "gpu": false,
        "gpuUsage": "N/A"
      },
      {
        "id": "9002",
        "startTime": "2024-03-10T10:56:02.025Z",
        "status": "UNLOADING",
        "memoryUsage": 0,
        "pid": 78929,
        "gpu": false,
        "gpuUsage": "N/A"
      },
      {
        "id": "9003",
        "startTime": "2024-03-10T10:56:02.025Z",
        "stat

In [32]:
! curl -X POST http://localhost:9080/predictions/pt_classifier \
        -H 'Content-Type: application/json' \
        -d '{"text":"herbal tea","request_id":"test_id"}' \

^C


In [35]:
!curl http://localhost:9080/ping

curl: (7) Failed to connect to localhost port 9080 after 0 ms: Couldn't connect to server


In [67]:
import requests

payload = {'text':"herbal tea", "request_ID": "test_id"}

endpoint = "http://localhost:9080/predictions/pt_classifier"

res = requests.post(endpoint, json = payload)

res.json()

{'code': 500, 'type': 'InternalServerException', 'message': 'Worker died.'}